In [1]:
from src.prepare_data import prepare_data, rotate_images
from imageai.Classification.Custom import ClassificationModelTrainer, CustomImageClassification


In [11]:
# Where the input images are located
PATH = f"images_final/"

# Directories for model
traindir = f"data/train"
validdir = f"data/val"
testdir = f"data/test"

In [8]:
rotate_images(PATH)

In [9]:
prepare_data(PATH, traindir, validdir, testdir,do_oversampling = False, oversampling =3)

mkdir: outputs: File exists


In [10]:
model_trainer = ClassificationModelTrainer()
model_trainer.setModelTypeAsResNet50()
model_trainer.setDataDirectory("./data")
model_trainer.trainModel(num_objects=5, num_experiments=100, enhance_data=True, batch_size=32, show_network_summary=True)

2021-12-08 00:01:43.386920: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-08 00:01:43.387243: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

2021-12-08 00:01:44.691888: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-12-08 00:01:44.691910: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-12-08 00:01:44.691976: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


Found 11757 images belonging to 5 classes.
Found 3919 images belonging to 5 classes.
JSON Mapping for the model classes saved to  ./data/json/model_class.json
Number of experiments (Epochs) :  100


/Users/macbook/Library/Python/3.8/lib/python/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-12-08 00:01:47.844193: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
367/367 [==============================] - 3742s 10s/step - loss: 2.0511 - accuracy: 0.2542 - val_loss: 1.6195 - val_accuracy: 0.2746

Epoch 00001: accuracy improved from -inf to 0.26362, saving model to ./data/models/model_ex-001_acc-0.263625.h5
Epoch 2/100
367/367 [==============================] - 3666s 10s/step - loss: 1.6321 - accuracy: 0.2832 - val_loss: 5.8876 - val_accuracy: 0.2897

Epoch 00002: accuracy improved from 0.26362 to 0.28213, saving model to ./data/models/model_ex-002_acc-0.282132.h5
Epoch 3/100
367/367 [==============================] - 3707s 10s/step - loss: 1.6019 - accuracy: 0.2799 - val_loss: 1.6780 - val_accuracy: 0.2323

Epoch 00003: accuracy improved from 0.28213 to 0.28324, saving model to ./data/models/model_ex-003_acc-0.283241.h5
Epoch 4/100
367/367 [==============================] - 3584s 10s/step - loss: 1.5832 - accuracy: 0.2863 - val_loss: 1.6256 - val_accuracy: 0.2974

Epoch 00004: accuracy improved from 0.28324 to 0.28657, saving model t

In [2]:
prediction = CustomImageClassification()
prediction.setModelTypeAsResNet50()
prediction.setModelPath('./data/models/model_ex-005_acc-0.287676.h5')
prediction.setJsonPath('./data/json/model_class.json')
prediction.loadModel(num_objects=5)

2021-12-08 22:08:25.714341: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-08 22:08:25.714564: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
predictions, probabilities = prediction.classifyImage('./data/test/fear/17_1027.jpg', result_count=5)

for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)

fear  :  100.0
sad  :  0.0
neutral  :  0.0
happy  :  0.0
angry  :  0.0


In [8]:
predictions, probabilities = prediction.classifyImage('./data/test/happy/7_4572.jpg', result_count=5)

for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)

fear  :  99.9566376209259
neutral  :  0.04336546408012509
sad  :  0.0
happy  :  0.0
angry  :  0.0


In [9]:
predictions, probabilities = prediction.classifyImage('./data/test/angry/77_5.jpg', result_count=5)

for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)

fear  :  99.99979734420776
neutral  :  0.00019766409877775004
sad  :  0.0
happy  :  0.0
angry  :  0.0


In [12]:
# Define the threshold
threshold = 0.5

categories = ['angry', 'fear', 'happy', 'neutral', 'sad']

preds = []
true = []

top1_true = 0
top2_true = 0
top3_true = 0
top4_true = 0
top5_true = 0

count = 0
#c = np.random.choice(self.categories)
for c in categories:
    root = testdir +"/"+ c + '/'
    imgs = os.listdir(root)
    for img in imgs:
        count = count +1 
        img_path = root + img

        classes, ps = prediction.classifyImage(img_path, result_count=5)
        #img, ps, classes, y_obs = nnc.predict(img_path, topk=5)
        # Check if we can predict
        if ps[0] > threshold:
            print(f"We predict {classes[0]} with {ps[0]*100} confidence%. true class was {c}")
            # Do stuff, such as moving to correct folder
            preds.append(classes[0])
            true.append(c)
        else:
            print("did not accuractely predict")
            #print(f"Couldn't confidently predict a single label, guiding user with this:")
            #nnc.display_prediction(img_selected, topk=5)
            # Maybe make a form for user to move to folder?

We predict fear with 10000.0 confidence%. true class was angry
We predict fear with 9787.94813156128 confidence%. true class was angry
We predict fear with 9737.415313720703 confidence%. true class was angry
We predict fear with 10000.0 confidence%. true class was angry
We predict neutral with 9997.45786190033 confidence%. true class was angry
We predict fear with 9961.114525794983 confidence%. true class was angry
We predict fear with 10000.0 confidence%. true class was angry
We predict fear with 9883.602857589722 confidence%. true class was angry
We predict fear with 10000.0 confidence%. true class was angry
We predict fear with 10000.0 confidence%. true class was angry
We predict neutral with 9999.72939491272 confidence%. true class was angry
We predict neutral with 9997.091889381409 confidence%. true class was angry
We predict fear with 10000.0 confidence%. true class was angry
We predict fear with 9998.936653137207 confidence%. true class was angry
We predict fear with 10000.0 con

In [16]:
a = preds
b = true

acc = sum(1 for x,y in zip(a,b) if x == y) / len(b)

print(f"Predicted on {len(a)} images ({100*len(a) / count})% of the dataset. got an accuracy of {100*acc}%")

3919
Predicted on 3919 images (100.0)% of the dataset. got an accuracy of 16.48379688696096%
